# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [1]:
from io import BytesIO
from datetime import datetime
import csv
from pathlib import Path

import requests
import pdfplumber

In [2]:
url = 'https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_5.1.2025.pdf'
req = requests.get(url)

In [3]:
pdf = pdfplumber.open(BytesIO(req.content))

In [4]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [5]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [6]:
lines

[['Statewide Report by County -May 1, 2025',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Democratic',
  'Libertarian',
  'Republican',
  'Independent',
  'OTH',
  'No Party\nAffiliation',
  'No Labels',
  'Inactive',
  'Total Active'],
 ['Aurora', '510', '2', '1,000', '232', '2', '100', '1', '102', '1,847'],
 ['Beadle',
  '2,479',
  '27',
  '5,204',
  '1,380',
  '40',
  '1,192',
  '1',
  '751',
  '10,323'],
 ['Bennett', '684', '7', '734', '201', '2', '202', '', '186', '1,830'],
 ['Bon Homme', '875', '12', '2,420', '491', '', '239', '', '283', '4,037'],
 ['Brookings',
  '5,087',
  '109',
  '10,100',
  '3,671',
  '21',
  '2,862',
  '1',
  '1570',
  '21,851'],
 ['Brown',
  '7,170',
  '102',
  '13,026',
  '3,588',
  '11',
  '1,994',
  '3',
  '1082',
  '25,894'],
 ['Brule', '884', '14', '1,768', '459', '4', '220', '', '165', '3,349'],
 ['Buffalo', '533', '2', '154', '76', '4', '101', '', '116', '870'],
 ['Butte', '770', '55', '4,642', '874', '21', '658', 

In [7]:
# report date is in the first line
report_date = lines.pop(0)

In [8]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2025-05-01


In [9]:
# save the PDF
outpath = Path('pdfs') / f'{report_date}.pdf'

with open(outpath, 'wb') as outfile:
    outfile.write(req.content)

print(f'Wrote {outpath}')

Wrote pdfs/2025-05-01.pdf


In [10]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [11]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [12]:
headers

['county',
 'democratic',
 'libertarian',
 'republican',
 'independent',
 'other',
 'npa',
 'no_labels',
 'inactive',
 'total active']

In [13]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [14]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [15]:
data[0]

{'county': 'Aurora',
 'democratic': 510,
 'libertarian': 2,
 'republican': 1000,
 'independent': 232,
 'other': 2,
 'npa': 100,
 'no_labels': 1,
 'inactive': 102,
 'date': '2025-05-01'}

In [16]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [17]:
# write to file
filepath = (Path('data') / f'{report_date}.csv').resolve()

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: /Users/cjwinchester/sd-voter-registration-data/data/2025-05-01.csv


In [18]:
%run build_files.py

Party list: americans_elect, constitution, democratic, inactive, independent, libertarian, no_labels, npa, npa_ind, npa_ind_oth, other, reform, republican
--------------------
Wrote sd-voter-registration-data.csv
Wrote sd-voter-registration-data-simplified.csv
Wrote README.md
